# Task 2.4 - Concentric sampled deformation gradients

## Setup

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['text.usetex'] = True
sns.set_style('darkgrid')

from keras import optimizers
from keras import losses

from src.models import CustomFFNN, InvariantsICNN
from src.data_import import load_train_test_concentric
from src.plots import plot_stress_predictions, plot_energy_prediction, plot_loss, plot_stress_currelation
from src.analytic_potential import get_C_features
from src.predict_utils import predict_multi_cases_PANN, predict_multi_cases_naive
from src.utils import get_scores

### Paths

In [ ]:
# Calibration paths
data_dir = os.path.abspath('concentric')

In [ ]:
def reached_good_prediction(labels: dict[str, np.ndarray], predictions: dict[str, np.ndarray], threshold: float = 1.0) -> bool:
    wmae = get_scores(labels, predictions, 'rmse', use_total=True)
    print(f'Weighted MAE: {max(list(wmae.values()))}')
    return all(abs(val < threshold) for val in wmae.values())

## Naive NN calibration on stress $P$

In [ ]:
def train_naive_model(train_F: tf.Tensor, train_P: tf.Tensor) -> CustomFFNN:
    naive_features = get_C_features(train_F)
    naive_labels = tf.reshape(train_P, (-1, 9))

    naive_model = CustomFFNN(
        hidden_sizes=[32, 32, 32, 9],
        activations=['softplus', 'softplus', 'softplus', 'linear']
    )
    naive_model.compile(
        optimizer=optimizers.Adam(learning_rate=0.01),
        loss=losses.MeanSquaredError()
    )
    naive_h = naive_model.fit(naive_features, naive_labels, batch_size=16, epochs=1000, verbose=0)
    naive_loss = naive_h.history['loss']
    plot_loss(naive_loss)
    return naive_model

In [ ]:
def predict_and_plot_naive_model(naive_model: CustomFFNN, examples_FPW_tup: dict[str, tuple[tf.Tensor, tf.Tensor, tf.Tensor]]):
    P_naive_test_labels, P_naive_test_preds = predict_multi_cases_naive(naive_model, examples_FPW_tup)
    plot_stress_predictions(P_naive_test_labels, P_naive_test_preds)

In [ ]:
def predict_and_correlate_naive_model(
        naive_model: CustomFFNN, 
        examples_FPW_tup: dict[str, tuple[tf.Tensor, tf.Tensor, tf.Tensor]]
) -> bool:
    P_naive_test_labels, P_naive_test_preds = predict_multi_cases_naive(naive_model, examples_FPW_tup)
    can_predict = reached_good_prediction(P_naive_test_labels, P_naive_test_preds)
    if can_predict:
        plot_stress_currelation(P_naive_test_labels, P_naive_test_preds)
    return can_predict

## PANN calibration on stress $P$ and energy $W$

In [ ]:
def train_pann_model(train_F: tf.Tensor, train_P: tf.Tensor, train_W: tf.Tensor) -> InvariantsICNN:
    pann_features = train_F
    pann_labels = (train_W, train_P)

    pann_model = InvariantsICNN(
        hidden_sizes=[16, 1],
        activations=['softplus', 'linear'],
        use_output_and_derivative=True
    )
    pann_model.compile(
        optimizer=optimizers.Adam(learning_rate=0.01),
        loss=losses.MeanSquaredError()
    )
    pann_h = pann_model.fit(pann_features, pann_labels, batch_size=32, epochs=1000, verbose=0)
    pann_loss = pann_h.history['loss']
    plot_loss(pann_loss)
    return pann_model

In [ ]:
def predict_and_plot_pann_model(pann_model: InvariantsICNN, examples_FPW_tup: dict[str, tuple[tf.Tensor, tf.Tensor, tf.Tensor]]):
    (
        P_pann_test_labels, W_pann_test_labels, P_pann_test_preds, W_pann_test_preds
    ) = predict_multi_cases_PANN(pann_model, examples_FPW_tup)
    plot_stress_predictions(P_pann_test_labels, P_pann_test_preds)
    plot_energy_prediction(W_pann_test_labels, W_pann_test_preds)

In [ ]:
def predict_and_correlate_pann_model(
        pann_model: InvariantsICNN, 
        examples_FPW_tup: dict[str, tuple[tf.Tensor, tf.Tensor, tf.Tensor]]
) -> bool:
    (P_pann_test_labels, _, P_pann_test_preds, _) = predict_multi_cases_PANN(pann_model, examples_FPW_tup)
    can_predict = reached_good_prediction(P_pann_test_labels, P_pann_test_preds) 
    if can_predict:
        plot_stress_currelation(P_pann_test_labels, P_pann_test_preds)
    return can_predict

## Load Datasets, calibrate and predict both models

In [ ]:
for test_size in [0.9, 0.75, 0.6, 0.5, 0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1]:
    print('=' * 100)
    print(f'Test size of {test_size*100}%')
    train_data, test_data = load_train_test_concentric(data_dir, test_size=test_size)

    train_F = tf.concat([tup[0] for tup in train_data.values()], axis=0)
    train_P = tf.concat([tup[1] for tup in train_data.values()], axis=0)
    train_W = tf.concat([tup[2] for tup in train_data.values()], axis=0)

    naive_model = train_naive_model(train_F, train_P)
    can_naive_pred = predict_and_correlate_naive_model(naive_model, test_data)

    pann_model = train_pann_model(train_F, train_P, train_W)
    can_pann_pred = predict_and_correlate_pann_model(pann_model, test_data)

    rand_examples = np.random.choice(list(test_data.keys()), size=6)
    example_test_FPW_tup = {f'Example {idx}': test_data[idx] for idx in rand_examples}
    predict_and_plot_pann_model(pann_model, example_test_FPW_tup)
    predict_and_plot_naive_model(naive_model, example_test_FPW_tup)

    if can_naive_pred and can_pann_pred:
        print(f'Able to predict with a test size of {test_size}')
        break